## 1. Imports

In [2]:
import pandas as pd
import glob
import os
import json

## 2. Load Datasets

#### 2.1 Load global games by year dataset
This data contains...

In [3]:
# Path to games_by_year folder
folder_path = './data/global_games_by_year/'

# get all files
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Read and combine the CSV's
df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

# Combine all into DataFrame
games_df = pd.concat(df_list, ignore_index=True)

# Show head
games_df.head()


,id,slug,name,released,metacritic,rating,ratings_count,playtime,added,platforms,genres,year
0,58172,spacewar-2,Spacewar!,1962-01-01,NaN,3.79,22,1,40,NaN,Arcade,1962
1,379437,zeusophthekroos,ZeusOfTheCrows,1970-01-01,NaN,0.00,4,0,16,NaN,Indie,1970
2,56681,the-oregon-trail-1971,The Oregon Trail (1971),1971-12-03,NaN,3.62,34,0,87,"PC, Classic Macintosh, Apple II, Commodore / A...","Adventure, Simulation",1971
3,55012,empire,Empire,1971-01-01,NaN,3.43,6,0,31,"PC, Commodore / Amiga, Atari ST",Strategy,1971
4,58174,computer-space,Computer Space,1971-12-01,NaN,2.91,8,0,29,NaN,Arcade,1971


#### 2.2 Load the top games for each company
This data contains the most popular games for each company, as wel as some more information.

In [4]:
# Path to company top games
json_folder = './data/company top games/'

# get all files
json_files = glob.glob(os.path.join(json_folder, '*.json'))

# List to hold all game entries
company_games_list = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
        # Only process files with actual game results
        if 'results' in data and isinstance(data['results'], list):
            # Extract company name from filename
            filename = os.path.basename(file)
            company_name = filename.replace('_top_games.json', '').replace('_', ' ')
            
            for game in data['results']:
                game['company'] = company_name
                company_games_list.append(game)

# Convert to DataFrame
company_games_df = pd.json_normalize(company_games_list)

# Get the first 5 unique companies
first_5_companies = company_games_df['company'].unique()[:5]

# Filter the DataFrame to only show rows from these companies
top_5_company_games = company_games_df[company_games_df['company'].isin(first_5_companies)]

# Show a preview of their games
top_5_company_games[['company', 'name', 'released', 'rating']].sort_values(by='company').head(20)

,company,name,released,rating
0,343 Industries,Halo: The Master Chief Collection,2014-11-11,4.25
1,343 Industries,Halo 2: Anniversary,2014-11-11,4.21
2,343 Industries,Halo: Combat Evolved Anniversary,2011-11-15,3.97
3,343 Industries,Halo 4,2012-11-06,3.82
4,343 Industries,Halo 5: Guardians,2015-10-27,3.74
5,343 Industries,Halo Wars: Definitive Edition,2017-04-20,3.72
6,343 Industries,Halo Infinite,2021-12-08,3.65
7,343 Industries,Halo: Spartan Strike,2015-04-16,3.28
8,343 Industries,Halo: Spartan Assault,2013-07-18,3.12
9,343 Industries,Halo Waypoint,2010-09-09,2.72


#### 2.3 Load the top 50 games per genre
This data contains...

In [5]:
# Path to top 50 games per genre
genre_folder = './data/top 50 games per genre/'
genre_files = glob.glob(os.path.join(genre_folder, '*.json'))

# List to hold all game data
genre_games_list = []

for file in genre_files:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)

        # Extract genre from the first genre tag in each game
        for game in data:
            if 'genres' in game and isinstance(game['genres'], list) and len(game['genres']) > 0:
                main_genre = game['genres'][0].get('name', 'Unknown')
            else:
                main_genre = 'Unknown'

            game['main_genre'] = main_genre
            genre_games_list.append(game)

# Convert to DataFrame
genre_games_df = pd.json_normalize(genre_games_list)

# Preview
genre_games_df[['main_genre', 'name', 'released', 'rating']].head()

,main_genre,name,released,rating
0,Action,Grand Theft Auto V,2013-09-17,4.47
1,Action,The Witcher 3: Wild Hunt,2015-05-18,4.65
2,Action,Tomb Raider (2013),2013-03-05,4.06
3,Action,Portal,2007-10-09,4.50
4,Shooter,Left 4 Dead 2,2009-11-17,4.09


#### 2.4 Load the top 5000 critic reviewed games
This data contains...

In [6]:
# Path to the file (adjust if needed)
file_path = './data/top 5000 critic reviewed games/top_5000_critic_reviewed_games.json'

# Load the JSON data
with open(file_path, 'r', encoding='utf-8') as f:
    critic_data = json.load(f)

# Convert to DataFrame
critic_df = pd.json_normalize(critic_data)

# Preview
critic_df[['name', 'released', 'metacritic', 'rating', 'genres']].head()

,name,released,metacritic,rating,genres
0,The Legend of Zelda: Ocarina of Time,1998-11-21,99,4.39,"[{'id': 4, 'name': 'Action', 'slug': 'action',..."
1,Soulcalibur (1998),1998-07-30,98,0.00,"[{'id': 6, 'name': 'Fighting', 'slug': 'fighti..."
2,Soulcalibur,1998-07-30,98,4.37,"[{'id': 4, 'name': 'Action', 'slug': 'action',..."
3,Baldur's Gate III,2023-08-03,97,4.47,"[{'id': 3, 'name': 'Adventure', 'slug': 'adven..."
4,Metroid Prime,2002-11-17,97,4.37,"[{'id': 4, 'name': 'Action', 'slug': 'action',..."


#### 2.5 Load detailed games by year
This data contains...

In [19]:
# Path to detailed_games_by_year
folder_path = './data/detailed_games_by_year'

# Empty list to store DataFrame
this_df = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # Extract the year from the filename
        year = int(filename.split("_")[-1].split(".")[0])
        df = pd.read_csv(file_path)
        df["year"] = year  # Add year column
        this_df.append(df)

# Combine all yearly data into one DataFrame
detailed_games_df = pd.concat(this_df, ignore_index=True)

# Display a preview
detailed_games_df.head()

,id,slug,name,released,metacritic,rating,ratings_count,playtime,added,twitch_count,youtube_count,description,website,reddit_url,platforms,genres,tags,developers,publishers,year
0,58172,spacewar-2,Spacewar!,1962-01-01,NaN,3.78,22,1,39,0,0,Spacewar! is a pioneering space combat video g...,NaN,NaN,NaN,Arcade,"2D, Retro, Space, 2-players",Steve Russell,Steve Russell,1962
1,379437,zeusophthekroos,ZeusOfTheCrows,1970-01-01,NaN,0.00,4,0,16,0,0,This should not be searcheable. If you find th...,NaN,NaN,NaN,Indie,NaN,NaN,NaN,1970
2,56681,the-oregon-trail-1971,The Oregon Trail (1971),1971-12-03,NaN,3.62,34,0,87,0,0,The Oregon Trail is a computer game originally...,NaN,NaN,"Commodore / Amiga, Classic Macintosh, Apple II...","Adventure, Simulation",Singleplayer,"MECC, Minnesota Educational Computing Consortium","Gameloft, Brøderbund, The Learning Company",1971
3,55012,empire,Empire,1971-01-01,NaN,3.43,6,0,31,0,0,Empire (or Classic Empire) is a turn-based war...,NaN,NaN,"Commodore / Amiga, Atari ST, PC",Strategy,"Singleplayer, Multiplayer","Walter Bright, Mark Baldwin, Peter Langston","New World Computing, Killer Bee Software, Inte...",1971
4,58174,computer-space,Computer Space,1971-12-01,NaN,2.91,8,0,29,0,0,Computer Space is a space combat arcade game d...,NaN,NaN,NaN,Arcade,"Space, combat, battle, history, rocket, colors",Syzygy Engineering,Nutting Associates,1971


## 3. Clean and Standardize Data
For each dataframe we:
- Drop irrelevant columns
- Rename columns for consistency
- Handle missing values
- Normalize strings

In [24]:
def normalize_title(df):
    df = df.rename(columns={"name": "title"})
    df["title"] = df["title"].str.lower().str.strip()
    return df

def extract_year_from_released(df):
    df["year"] = pd.to_datetime(df["released"], errors="coerce").dt.year
    return df

## 4. Add Metadata
(Examples of what we could do here:)
- Platform category (e.g., PC vs Console)
- Convert scores to numeric
- Tags or buckets like: decade

In [21]:
# TODO: add metadata

## 5. Merge the dataframes into on database

In [27]:
# Voor debugging:
# print("games_df columns:", games_df.columns)
# print("detailed_games_df columns:", detailed_games_df.columns)
# print("company_games_df columns:", company_games_df.columns)
# print("genre_games_df columns:", genre_games_df.columns)
# print("company_games_df columns:", company_games_df.columns)

# normalise the titles
games_df = normalize_title(games_df)
detailed_games_df = normalize_title(detailed_games_df)
company_games_df = normalize_title(company_games_df)
genre_games_df = normalize_title(genre_games_df)
platform_df = normalize_title(company_games_df)

# ensure year columns
company_games_df = extract_year_from_released(company_games_df)
genre_games_df = extract_year_from_released(genre_games_df)

# Rename for consistency
company_games_df = company_games_df.rename(columns={"name": "title"})
genre_games_df = genre_games_df.rename(columns={"name": "title"})
platform_df = platform_df.rename(columns={"name": "title"})

# Merge everything piece by piece
merge1 = pd.merge(games_df, detailed_games_df, on=["title", "year"], how="outer", suffixes=("_games", "_detailed"))
merge2 = pd.merge(merge1, company_games_df, on=["title", "year"], how="outer", suffixes=("", "_company"))
merge3 = pd.merge(merge2, genre_games_df, on=["title", "year"], how="outer", suffixes=("", "_genre"))
merged_all = pd.merge(merge3, platform_df, on=["title", "year"], how="outer", suffixes=("", "_platform"))

# Preview
merged_all.head()

,id_games,slug_games,title,released_games,metacritic_games,rating_games,ratings_count_games,playtime_games,added_games,platforms_games,...,added_by_status.toplay_platform,added_by_status.dropped_platform,added_by_status.playing_platform,esrb_rating.id_platform,esrb_rating.name_platform,esrb_rating.slug_platform,esrb_rating.name_en_platform,esrb_rating.name_ru_platform,community_rating_platform,added_by_status_platform
0,58172.0,spacewar-2,spacewar!,1962-01-01,NaN,3.79,22.0,1.0,40.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,379437.0,zeusophthekroos,zeusofthecrows,1970-01-01,NaN,0.00,4.0,0.0,16.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56681.0,the-oregon-trail-1971,the oregon trail (1971),1971-12-03,NaN,3.62,34.0,0.0,87.0,"PC, Classic Macintosh, Apple II, Commodore / A...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55012.0,empire,empire,1971-01-01,NaN,3.43,6.0,0.0,31.0,"PC, Commodore / Amiga, Atari ST",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58174.0,computer-space,computer space,1971-12-01,NaN,2.91,8.0,0.0,29.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 6. Save the final database

In [29]:
merged_all.to_csv("data/Final Database/merged_game_data.csv", index=False)